# Quantifying relationships between variables

In [ ]:
import six
from six.moves.urllib.request import urlretrieve
import landsat_sample_functions as lf

#landsat_RGBN.txt
rgb_url = "http://bit.ly/2uzjBOX"
rgbdatafile, _ = urlretrieve(rgb_url)
rgbn = lf.landsat_read(rgbdatafile)

In [ ]:
# extract colors
red = rgbn[...,0]
green = rgbn[...,1]
blue = rgbn[...,2]

#extract near infrared channel from rgb
nir = rgbn[...,3]

In [ ]:
# Let's compute vegetation! 
def NDVI(nir, red):
    return (nir-red)/(nir+red)

ndvi = NDVI(nir, red)

In [ ]:
# let's look at those components 

%matplotlib inline
import matplotlib.pyplot as plt 

fig,(ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(10,30))

nim = ax1.imshow(nir)
#fraction shrinks the colorbar so that it fits nicely
fig.colorbar(nim, ax=ax1, fraction=.04)
ax1.set_title("Near Infrared")

rim = ax2.imshow(red, cmap="Reds")
fig.colorbar(rim, ax=ax2, fraction=.04)
ax2.set_title("red channel")

ndim = ax3.imshow(ndvi)
fig.colorbar(ndim, ax=ax3, fraction=.04)

fig.tight_layout()

# Review
Compute and plot the probability density estimates of:
1. near infrared data
2. red channel data
2. ndvi data

Scales are totally different, so how do we see if there's a relationship? 
Scatter plots!

Because the images are the same size, the pixels are at the same latitude and longitude and that relationship is preserved even when the data is flattened. 

In [ ]:
fig, ax = plt.subplots()

#scatter puts the nir data on the x axis, temperature on y
ax.scatter(nir.flatten(), ndvi.flatten())

ax.set(xlabel = "NIR", ylabel="NDVI")
fig.show()

In [ ]:
# Let's make those dots smaller

fig, ax = plt.subplots()

#scatter puts the nir data on the x axis, temperature on y
ax.scatter(nir.flatten(), ndvi.flatten(), s=.25)

ax.set(xlabel = "NIR", ylabel="NDVI")
fig.show()

In [ ]:
# Can we fit a line?

import scipy.stats as st

x = nir.flatten()
y = ndvi.flatten()
slope, intercept, r_value, p_value, std_err = st.linregress(x, y)

In [ ]:
# how do we plot that? Equation of line: y = m*x + b 
# let's compute theoretical line
# we sort x since y is a line 
predicted_y = slope*sorted(x) + intercept

In [ ]:
fig, ax = plt.subplots()


ax.scatter(x, y, s=.25)

ax.plot(sorted(x), possible_y, color="black")

ax.set(xlabel = "NIR", ylabel="NDVI")
fig.show()

In [ ]:
# Is it meaningful? 
# r_value = Pearson's r correlation coefficient 
# p_value = likelihood null hypothesis is true
# std_error = avg distance between predicted and actual value
r_value, p_value, std_err

# To do

See if the red channel is correlated with 
NDVI. Draw a scatter plot with a best fit line.

# Let's add in temperature

In [ ]:
#landsat_thermrad.txt
radiance_url = "http://tinyurl.com/ya5bz7ue"
raddatafile, _ = urlretrieve(radiance_url)
thermrad = lf.landsat_read(raddatafile)

In [ ]:
# convert thermal radiance to temperature
temps = lf.BT(thermrad,10.9,1)

# To do
1. Plot the temperature data
2. Compute the probability density of the temperature values
3. Plot the temperature against the NDVI, compute the best fit line, and see if there's a correlation

# Is there another way to see vegetation?

In [ ]:
#Global Agriculture: % of land cover (land_cover.csv)
url = "http://tinyurl.com/y8bajr5f"

In [ ]:
#let's use pandas to open the file
import pandas as pd
df = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
df

In [ ]:
# let's only see the first few
df.head(5)

In [ ]:
# columns? 
df.columns

In [ ]:
# Why is this useful
df.describe()

In [ ]:
# What if we just want the US, Canada and Mexico?
df[["United States", "Canada", "Mexico"]].describe()

# To do
Get the stats for any other 2 countries

In [ ]:
# How about the whole data set, for one country?
df["United States"]

In [ ]:
# plot that over time?
%matplotlib inline
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
df["United States"].plot(ax=ax)
ax.set(ylabel="% land is agricultural")

In [ ]:
# more countries?
fig, ax = plt.subplots()
df[["United States", "Mexico"]].plot(ax=ax)
ax.set(ylabel="% land is agricultural")

# To do
Add another 2 countries to the above plot


In [ ]:
# What about histograms...
df["United States"].hist(bins=100)

In [ ]:
# And densities?
df["United States"].plot.density()

In [ ]:
# both?
fig, ax = plt.subplots()
df["United States"].hist(bins=100, normed=True, ax=ax)
df["United States"].plot.density()

In [ ]:
# scatters?
df.plot.scatter("United States", "Mexico")

In [ ]:
# First compute the best fit line

slope, intercept, r_value, p_value, std_err = st.linregress(
                                            df["United States"],
                                            df["Mexico"])
predicted_y = slope*df["United States"].sort_values() + intercept

In [ ]:
fig, ax = plt.subplots()

ax.scatter(df["United States"], df["Mexico"])

ax.plot(df["United States"].sort_values(), predicted_y, color="black")

ax.set(xlabel="United States", ylabel="Mexico")

In [ ]:
r_value, p_value, std_err

# To Do
Repeat with two different countries

In [ ]:
# Can this be automated kinda too?
df[["United States", "Canada", "Mexico"]].corr()

# To do
Choose 5 countries and find out how they correlate to each other.

In [ ]:
# What if I want all the countries but just a year?
# data frames are usually indexed via column then row
# loc uses numpy/python row then column slicing
df.loc['2012-01-01']

In [ ]:
# visualize that?
fig, ax = plt.subplots()
df[["United States", "Canada", "Mexico"]].loc['2012-01-01'].plot.bar(ax=ax)
ax.set(ylabel="% land cover")
fig.show()

# To do:
* Plot the graph for the first year in the dataset
* Plot the graph for any other year
* Choose any other 4 countries and plot the bar graph for them

In [ ]:
# can we do table?
table = df[["United States", "Canada", "Mexico"]].loc['2000':'2012']
table

In [ ]:
table.plot()

To Do
===
Pick some other countries and years. See if you can find rapid urbanization or some other trends...